## A Numerical Solver for the Compact Domains Problem

### Review of the Problem

This Jupyter notebook accompanies Section 4.5 of my PhD Thesis, and provides a method to explicitly compute the dual supremum $\mathcal{S}_{X,Y}(\mu,m)$ and the primal infimum $\mathcal{I}_{X,Y}(\mu,m)$ where $Y=[0,R]^2$. It also allows us to compute the primal optimizer $\gamma_0$ and the dual optimizer $(\varphi^{\lambda_0},\lambda_0).$

First, import some libraries:

In [123]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)

Next, we implement Equation (4.6) in Proposition 4.13 of my thesis which, given $x\in [0,\infty)\times (0,\infty),$ $\lambda\in \R^2$, and $R>0$, allows us to compute $$Y(x;\lambda)\in \operatorname{argmin}_{y\in[0,R]^2} [c(x,y)-\varphi(x)-\lambda\cdot y].$$
For the purpose of simplifying the numerics, we assume that $Y(x;\lambda)=(0,0)$ in the case where  $\operatorname{argmin}_{y\in[0,R]^2} [c(x,y)-\varphi(x)-\lambda\cdot y]$ is a line segment; it should be noted that this may affect whether the resulting measure $\gamma$ satisfies the mean constraint $\int y\; d\gamma(x,y)=m.$ 

In [ ]:
# Start by defining the auxiliary functions
def func_Lda1(x1 : float, x2: float, lda1 : float) -> float:
    return  x1 +  x2 * lda1 

def func_Lda2(x1 : float, x2: float, lda2 : float) -> float:
    return x1**2 - 2 * x2 * lda2 

# Next, define the cost function
def func_c(x1: float, x2: float, y1: float, y2: float) -> float:
    if (x2 > 0) and (y2 > 0):
        return (y2/(2*x2))*(x1-y1/y2)**2
    elif x1 * y2 - y1 == 0:
        return 0
    else: 
        return float('inf') 
    
# Compute $Y$ as in Equation 4.6.
def func_Y(x1 : float, x2: float, lda1 : float, lda2 : float, R : float) -> np.ndarray:
    Lda1 = func_Lda1(x1, x2, lda1)
    Lda2 = func_Lda2(x1, x2, lda2)
    if (Lda1 <= 0) and (Lda2 < 0):
        return np.array([0,R])
    elif (0 <= Lda1 <=1) and (Lda2 < Lda1**2):
        return R * np.array([Lda1,1])
    elif (Lda2 <= 1 <= Lda1):
        return np.array([R,R])
    elif (Lda1 >= 1) and (1 <= Lda2 < Lda1**2):
        return R*np.array([1, 1/np.sqrt(Lda2)])
    else:
        return np.array([0,0])
    

Now, we define a measure and compute key quantities. We encode discrete probability measures as `numpy` `ndarrays`, where each row has three entries -- the mass, the $x_1$ coordinate, and the $x_2$ coordinate. We also set a mean target and a variance target.

In [ ]:
# Define one sample measure for which the dual supremum appears to be attained
mu_denorm_attain = np.array([[1, 1, 1], [1, 2, 1], [1, 1, 2], [1, 2,2]], dtype = 'f')
# Define another sample measure for which the dual supremum appears not to be attained. 
mu_denorm_nonattain = np.array([[1, 2, 4]], dtype = 'f')
# Automatically normalize both measures
mu_norm_attain = mu_denorm_attain
mu_norm_attain[:,0] = mu_norm_attain[:,0]/(np.sum(mu_norm_attain[:,0], axis = 0))
mu_norm_nonattain = mu_denorm_nonattain
mu_norm_nonattain[:,0] = mu_norm_nonattain[:,0]/(np.sum(mu_norm_nonattain[:,0], axis = 0))

# Set mean target
m = np.array([1,2])
# Set variance target
tau = 1.5
# Set size of $Y$


The choice of $Y=[0,R]^2$ along with the presence of a prescribed mean, allows us to compute $\varphi^\lambda$ (which is the optimal choice of $\varphi$ admissible for the dual supremum) as in Equation (4.7):


In [ ]:
def phi_lambda(x1: float, x2: float, lda1: float, lda2: float, R : float) -> float:
    Lda1 = func_Lda1(x1, x2, lda1)
    Lda2 = func_Lda2(x1, x2, lda2)
    if (Lda1 <= 0) and (Lda2 <= 0):
        return (R * Lda2) / (2 * x2)
    elif (0 <= Lda1 <=1) and (Lda2 <= Lda1**2):
        return R*(Lda2 - Lda1**2)/(2 * x2)
    elif (Lda2 <= 1 <= Lda1):
        return R*(Lda2 - 2 * Lda1 + 1)/(2 * x2)
    elif (Lda1 >= 1) and (1 <= Lda2 <= Lda1**2):
        return R*(np.sqrt(Lda2) -  Lda1)/(x2)
    else:
        return 0


Next, we define a function to compute the dual objective function 
$$\int \varphi\; d\mu+\lambda\cdot y.$$

In [ ]:
def objective_function(params, R, measure) -> float:
    lda1, lda2 = params
    # negative because scipy can only minimize
    return (-sum([measure[i,0] * phi_lambda(measure[i, 1], measure[i, 2], lda1, lda2, R) for i in range(len(measure))])
            - lda1 * m[0] - lda2 * m[1])

We will also wish to compute the mean $$\int Y(\cdot;\lambda,\eta)\; d\mu$$ to verify if our result actually achieves the mean constraint:

In [ ]:
def mu_mean(params, R, measure) -> np.ndarray:
    lda1, lda2 = params
    return sum([measure[i,0] * func_Y(measure[i,1], measure[i,2], lda1, lda2, R) for i in range(len(measure))])

Moreover, we compute $$\int c(x, Y(x;\lambda,\eta))d\mu(x)$$ in order to get a sense of the value of the primal infimum.

In [ ]:
def primal_cost(params, R, measure) -> np.ndarray:
    lda1, lda2 = params
    return sum([measure[i,0] * func_c(measure[i,1], measure[i,2], 
                                      func_Y(measure[i,1], measure[i,2], lda1, lda2, R)[0],
                                      func_Y(measure[i,1], measure[i,2], lda1, lda2, R)[1]) 
                                      for i in range(len(measure))])

Finally, for technical reasons, we also create a wrapper so that we can use SciPy's optimize function. 

In [130]:
def make_objective(measure):
    def wrapped_objective(params):
        return objective_function(params, measure)
    return wrapped_objective

Next, we finally complete the optimization:

In [ ]:
optimization_result_attain = opt.minimize(make_objective(mu_norm_attain), [-1,1], method = 'Nelder-Mead')

We recover the optimal choice of $\lambda.$ 

In [ ]:
optimal_params_attain = optimization_result_attain.x
print("lambda = " +  str(optimal_params_attain))

lambda = [-0.47282743  0.54774745]
eta = -0.09954221634326921


Next, we verify that the result achieves the target mean and, which is the case for `mu_norm_attain`. We also compute the value of the objective function, which in this case should be approximately equal to the dual supremum. 

In [ ]:
print("mean: " +  str(mu_mean(optimal_params_attain, mu_norm_attain)))
print("dual value: " + str(-objective_function(optimal_params_attain, mu_norm_attain)))
print("primal value: " + str(primal_cost(optimal_params_attain, mu_norm_attain)))

mean: [0.99999678+0.j 2.00002156+0.j]
variance: 1.5000480053980814
dual value: (0.32402810105060625+0j)
primal value: (0.3240366522395013+0j)
